In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score



In [7]:
#We shall begin by importing the 2 US bureau dataframes from each year that we are concerned with:

df1 = pd.read_excel("/Users/jacob/Documents/4YP data/US_BureaOfLaborStatistics/annualstats/national_M2019_dl.xlsx")
df2 = pd.read_excel("/Users/jacob/Documents/4YP data/US_BureaOfLaborStatistics/annualstats/national_M2014_dl.xlsx")


In [10]:
#We shall now drop the unimportant columns:

df1 = df1[["occ_code","occ_title","tot_emp"]]
df1 = df1.rename(columns={"occ_code": "OCC_CODE", "occ_title": "OCC_TITLE", "tot_emp": "TOT_EMP"}) #ensure consistency of labels
df2 = df2[["OCC_CODE","OCC_TITLE","TOT_EMP"]]


In [11]:
#We shall now focus on concatenating these dataframes

def concat_data(a,b,key):
    for i in list(b["OCC_CODE"]):
        a.loc[a["OCC_CODE"] == i,key] = list(b.loc[b["OCC_CODE"] == i,key])

concat_data(df1,df2,"TOT_EMP")
df1.head()

ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
def title_set(my_string):
    my_list = []
    my_list[:0] = my_string
    my_list.remove("_")
    my_list.append(".00")
    my_output = "".join(my_list)
    return my_output

